In [1]:
%matplotlib inline
import pandas as pd
import librosa as lib
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Audio
import scipy
import pyarrow.feather as feather

In [2]:
#import motion dataframe of only participant 1
motion1 = feather.read_feather('Participant_Motion/PID_motion_1')

Reminder of what each look like

In [3]:
motion1.head(), motion1.shape

(         X         Y         Z  PID
 0  159.596  1845.594  1732.156    1
 1  159.583  1845.451  1732.144    1
 2  159.584  1845.308  1732.126    1
 3  159.587  1845.180  1732.105    1
 4  159.597  1845.072  1732.103    1,
 (36001, 4))

In [4]:
motion1.describe()

,X,Y,Z,PID
count,36001.000000,36001.000000,36001.000000,36001.0
mean,160.456958,1860.184987,1724.088614,1.0
std,4.317600,11.164595,6.404407,0.0
min,144.325000,1829.400000,1715.278000,1.0
25%,158.102000,1852.813000,1718.380000,1.0
50%,161.018000,1859.077000,1721.902000,1.0
75%,163.689000,1866.659000,1732.118000,1.0
max,170.325000,1901.478000,1733.874000,1.0


In [5]:
time = pd.date_range(start='00:00', freq='10L', periods=36001)
time.dtype

dtype('<M8[ns]')

In [6]:
time = time.strftime('%M:%S.%f')
time.dtype

dtype('O')

In [7]:
motiont = motion1.set_index(time)

In [8]:
motiont.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36001 entries, 00:00.000000 to 06:00.000000
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       36001 non-null  float64
 1   Y       36001 non-null  float64
 2   Z       36001 non-null  float64
 3   PID     36001 non-null  int64  
dtypes: float64(3), int64(1)
memory usage: 1.4+ MB


In [9]:
motiont['segment'] = 'Silence'
motiont.loc['00:03:00':'00:03:20', 'segment'] = 'class1'
motiont.loc['03:20':'03:35', 'segment'] = 'class2'
motiont.loc['03:35':'04:15', 'segment'] = 'class3'
motiont.loc['04:15':'04:35', 'segment'] = 'class4'
motiont.loc['04:35':'05:10', 'segment'] = 'hiphop'
motiont.loc['05:10':'05:31', 'segment'] = 'salsa'
motiont.loc['05:31':, 'segment'] = 'edm'

In [10]:
motiont['segment']

00:00.000000    Silence
00:00.010000    Silence
00:00.020000    Silence
00:00.030000    Silence
00:00.040000    Silence
                 ...   
05:59.960000        edm
05:59.970000        edm
05:59.980000        edm
05:59.990000        edm
06:00.000000        edm
Name: segment, Length: 36001, dtype: object

In [11]:
motiont.index = pd.to_datetime(motiont.index, format='%M:%S.%f')

In [12]:
motiont.shape

(36001, 5)

In [13]:
#set the music dataframe index to a time series and upsamplt to match the 100 readings per second from the motion data
motionr = motiont.resample('200L').mean()

In [14]:
motionr#.resample('20L').mean()

,X,Y,Z,PID
1900-01-01 00:00:00.000,159.69750,1844.71755,1732.09535,1
1900-01-01 00:00:00.200,160.21170,1844.63625,1732.06770,1
1900-01-01 00:00:00.400,160.53550,1846.72830,1732.12305,1
1900-01-01 00:00:00.600,160.62530,1849.30315,1732.22560,1
1900-01-01 00:00:00.800,160.51940,1850.78495,1732.32555,1
...,...,...,...,...
1900-01-01 00:05:59.200,156.83400,1833.71825,1715.62615,1
1900-01-01 00:05:59.400,157.20980,1832.30235,1715.69770,1
1900-01-01 00:05:59.600,158.44670,1833.38550,1715.61895,1
1900-01-01 00:05:59.800,159.71515,1836.19990,1715.66870,1


### Matching Times Between DataFrames
We need to match the musics and motion DataFrames together. We will create a 'time' column that repersents hundreths of seconds from the start of the motion capture. The music starts at 3 minutes or 18000 hundreths of a second

In [18]:
motion1['time(hs)'] = list(range(0,36001))

In [19]:
motion1

,X,Y,Z,PID,time(hs)
0,159.596,1845.594,1732.156,1,0
1,159.583,1845.451,1732.144,1,1
2,159.584,1845.308,1732.126,1,2
3,159.587,1845.180,1732.105,1,3
4,159.597,1845.072,1732.103,1,4
...,...,...,...,...,...
35996,160.090,1837.312,1715.629,1,35996
35997,160.122,1837.488,1715.619,1,35997
35998,160.128,1837.653,1715.597,1,35998
35999,160.147,1837.829,1715.599,1,35999


In [74]:
def Import_Participant_Motion(par_list):
    #import each participant motion data from the file, add datetime index and resample to 2 tenths of a second
    #output dictionary of dataframes of motion
    output = {}
    time = pd.date_range(start='00:00', freq='10L', periods=35601)
    time = time.strftime('%M:%S.%f')
    for participant in par_list:
        motiont = feather.read_feather('Participant_Motion/' + participant)
        motiont = motiont.iloc[:35601].set_index(time)
        motiont.index = pd.to_datetime(motiont.index, format='%M:%S.%f')
        motionr = motiont.resample('200L').mean()
        motionr = motionr.reset_index()
        output[participant] = motionr
    return output

There is some variation for the total length of motion data across the participants. We have limitied the data to the smallest value which cuts off the last 4 seconds of the recordings so as to cause bias by using data at the end of the song that is only from a limited section of participants

In [21]:
import os

In [67]:
par_motion_list = os.listdir('Participant_Motion')


In [94]:
motion_dict = Import_Participant_Motion(par_motion_list)

In [89]:
motion_dict['PID_motion_10']

,index,X,Y,Z,PID
0,1900-01-01 00:00:00.000,983.71885,1798.82115,1773.37345,10
1,1900-01-01 00:00:00.200,982.49355,1799.03930,1773.41555,10
2,1900-01-01 00:00:00.400,980.76175,1798.55955,1773.43950,10
3,1900-01-01 00:00:00.600,979.80755,1796.75025,1773.54110,10
4,1900-01-01 00:00:00.800,980.24215,1796.71095,1773.50675,10
...,...,...,...,...,...
1776,1900-01-01 00:05:55.200,984.35440,1825.30950,1768.39915,10
1777,1900-01-01 00:05:55.400,984.08315,1826.30870,1768.39805,10
1778,1900-01-01 00:05:55.600,983.66335,1827.15250,1768.37490,10
1779,1900-01-01 00:05:55.800,982.96215,1827.45955,1768.39455,10


In [95]:
bunch = motion_dict['PID_motion_1']
for participant in par_motion_list[1:]:
    bunch = pd.concat([bunch, motion_dict[participant]], join='outer')

In [97]:
bunch = bunch.sort_values(by='index')

In [99]:
motion_dict['PID_motion_24']

,index,X,Y,Z,PID
0,1900-01-01 00:00:00.000,1773.13555,966.17255,1705.14140,24
1,1900-01-01 00:00:00.200,1774.34025,966.03585,1704.72575,24
2,1900-01-01 00:00:00.400,1775.35690,965.23255,1704.61140,24
3,1900-01-01 00:00:00.600,1776.48505,967.20900,1704.31145,24
4,1900-01-01 00:00:00.800,1776.07525,970.53165,1704.15400,24
...,...,...,...,...,...
1776,1900-01-01 00:05:55.200,1779.35100,988.12345,1704.36105,24
1777,1900-01-01 00:05:55.400,1778.28195,987.17395,1704.62225,24
1778,1900-01-01 00:05:55.600,1777.61850,988.02800,1704.57015,24
1779,1900-01-01 00:05:55.800,1776.96410,989.37175,1704.34535,24
